In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

device = torch.device("mps")

In [7]:
PATH = '/Users/rishinigam/t81_588_course/datasets/iris-image'

In [10]:
# transformation and data augmentation for training and validation
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(360),
    transforms.RandomResizedCrop(256, scale=(0.5, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder(root=PATH, transform=train_transforms)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = ImageFolder(root=PATH, transform=val_transforms)
val_dataloader = DataLoader(train_dataloader, batch_size=16, shuffle=False)

In [11]:
# modeling
class_cnt = len(train_dataset.classes)


3